In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Pipelines: Evaluating BatchPrediction results from an AutoML Tabular forecasting model

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_forecasting_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/model_evaluation/automl_tabular_forecasting_model_evaluation.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/model_evaluation/automl_tabular_forecasting_model_evaluation.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## Overview


This notebook demonstrates how to use the Vertex AI forecasting model evaluation component to evaluate an AutoML tabular forecasting model. Model evaluation helps you determine your model performance based on the evaluation metrics and improve the model if necessary.

### Objective

In this tutorial, you learn to use Vertex AI Pipelines and Google Cloud Pipeline Components to evaluate an AutoML tabular forecasting model.


This tutorial uses the following Google Cloud ML services:

- Vertex AI Datasets
- Vertex AI Training (AutoML Tabular Forecasting) 
- Vertex AI Model Registry
- Vertex AI Pipelines
- Vertex AI Batch Predictions

The steps performed include:

- Create a Vertex AI Dataset.
- Train a AutoML Tabular Forecasting model on the Dataset resource.
- Import the trained AutoML model resource into the pipeline.
- Run a Batch Prediction job.
- Evaulate the AutoML model using the Forecasting Evaluation Component.
- Import the Forecasting metrics to the AutoML model resource.


### Dataset

The dataset used for this tutorial a time series dataset containing samples drawn from the Iowa Liquor Retail Sales dataset. Data were made available by the Iowa Department of Commerce. It is provided under the Creative Commons Zero v1.0 Universal license. 
Learn more about the dataset from [Iowa Liquor Retail Sales](https://console.cloud.google.com/marketplace/product/iowa-department-of-commerce/iowa-liquor-sales). This dataset does not require any feature engineering. The version of the dataset that you use in this tutorial is stored in BigQuery.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* BigQuery

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

### Set up your local development environment

If you are using Colab or Workbench AI Notebooks, your environment already meets all the requirements to run this notebook. You can skip this step.

Otherwise, make sure your environment meets this notebook's requirements. You need the following:

- The Cloud Storage SDK
- Git
- Python 3
- virtualenv
- Jupyter notebook running in a virtual environment with Python 3

The Cloud Storage guide to [Setting up a Python development environment](https://cloud.google.com/python/setup) and the [Jupyter installation guide](https://jupyter.org/install) provide detailed instructions for meeting these requirements. The following steps provide a condensed set of instructions:

1. [Install and initialize the SDK](https://cloud.google.com/sdk/docs/).

2. [Install Python 3](https://cloud.google.com/python/setup#installing_python).

3. [Install virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv) and create a virtual environment that uses Python 3.  Activate the virtual environment.

4. To install Jupyter, run `pip3 install jupyter` on the command-line in a terminal shell.

5. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

6. Open this notebook in the Jupyter Notebook Dashboard.


## Installation

Install the latest version of Vertex AI SDK for Python.

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --upgrade google-cloud-aiplatform {USER_FLAG} -q
! pip3 install --upgrade google-cloud-pipeline-components==1.0.26 {USER_FLAG} -q
! pip3 install --upgrade matplotlib {USER_FLAG} -q
! pip3 install kfp --upgrade -q
! pip3 install --user --upgrade 'google-cloud-bigquery[bqstorage,pandas]'  -q

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

#### UUID

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a uuid for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### Authenticate your Google Cloud account

**If you are using Workbench AI Notebooks**, your environment is already authenticated. 

**If you are using Colab**, run the cell below and follow the instructions when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

In the Cloud Console, go to the [Create service account key](https://console.cloud.google.com/apis/credentials/serviceaccountkey) page.

**Click Create service account**.

In the **Service account name** field, enter a name, and click **Create**.

In the **Grant this service account access to project** section, click the Role drop-down list. Type "Vertex" into the filter box, and select **Vertex Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

Click Create. A JSON file that contains your key downloads to your local environment.

Enter the path to your service account key as the GOOGLE_APPLICATION_CREDENTIALS variable in the cell below and run the cell.

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = f"gs://{BUCKET_NAME}"

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_URI

#### Service Account

You use a service account to create Vertex AI Pipeline jobs. If you do not want to use your project's Compute Engine service account, set `SERVICE_ACCOUNT` to another service account ID.

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import os
import sys

if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

#### Set service account access for Vertex AI Pipelines

Run the following commands to grant your service account access to read and write pipeline artifacts in the bucket that you created in the previous step. You only need to run this step once per service account.

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

### Import libraries

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud.bigquery import Client
from kfp.v2 import compiler

DATASET_ID = "samples" + "_" + UUID
TABLE_NAME = "2021_sales_predict"

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

## Train an AutoML Tabular forecasting model

Now you are ready to start creating your own AutoML tabular forecasting model.

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the Vertex AI's `TimeSeriesDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `bq_source`: Alternatively, import data items from a BigQuery table into the `Dataset` resource.

This operation may take several minutes.

In [ ]:
dataset = aiplatform.TimeSeriesDataset.create(
    display_name="iowa_liquor_sales_train" + "_" + UUID,
    bq_source="bq://bigquery-public-data.iowa_liquor_sales_forecasting.2020_sales_train",
)

time_column = "date"
time_series_identifier_column = "store_name"
target_column = "sale_dollars"

print(dataset.resource_name)

### Create Prediction data with ground_truth input

For evaluating your forecast model, you create your BigQuery table with a ground truth column named `ground_truth_sale_dollars`. This field consists of the original sales values. On the other hand, the original sales column i.e. `sale_dollars` is kept 50% null to make room for forecasts.

Learn more about [predictions from a forecast model on Vertex AI](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/get-predictions).

**Create a BigQuery dataset**

In [ ]:
client = Client(project=PROJECT_ID)

query = """
CREATE SCHEMA `{PROJECT_ID}.{DATASET_ID}`
OPTIONS(
  location="us"
  )
""".format(
    PROJECT_ID=PROJECT_ID, DATASET_ID=DATASET_ID
)
query_job = client.query(query)
print(query_job.result())

In [ ]:
# %%bigquery df_2021_sales_predict


query = """
CREATE or REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.{TABLE_NAME}` as
with store_count as (
  select store_name,count(1) as cnt
  from `bigquery-public-data.iowa_liquor_sales_forecasting.2021_sales_predict`
  where sale_dollars is not null and county is not null
  group by store_name
  having count(1)>1
)
select 
  date,
  a.store_name,
  city,
  zip_code,
  county,
  sale_dollars as ground_truth_actual_sales_dollars,
  case when row_number() over (partition by a.store_name order by date) <= round(b.cnt/2) then sale_dollars else null end as sale_dollars,
  # row_number() over (partition by a.store_name order by date) as R_NUM,
  # b.cnt as Store_Count  
from `bigquery-public-data.iowa_liquor_sales_forecasting.2021_sales_predict` a
inner join store_count b
on a.store_name=b.store_name
where sale_dollars is not null and county is not null
order by a.store_name; 
""".format(
    PROJECT_ID=PROJECT_ID, DATASET_ID=DATASET_ID, TABLE_NAME=TABLE_NAME
)

query_job = client.query(query)
print(query_job.result())

#### Describe the column specifications

In [ ]:
COLUMN_SPECS = {
    time_column: "timestamp",
    target_column: "numeric",
    "city": "categorical",
    "zip_code": "categorical",
    "county": "categorical",
}

To train an AutoML model, you perform two steps: 
1) Create a training job, and 
2) Run the job.

### Create the training job

An AutoML training job is created with the AutoMLForecastingTrainingJob class, using the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `column_transformations`: (Optional): Transformations to apply to the input columns
- `optimization_objective`: The optimization objective to minimize or maximize.
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

The instantiated object is the job for the training pipeline.

In [ ]:
MODEL_DISPLAY_NAME = f"iowa-liquor-sales-forecast-model_{UUID}"

training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=MODEL_DISPLAY_NAME,
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

### Run the training Job

Next, you start the training job by invoking the `run` method, with the following parameters:

- `dataset`: The Vertex AI `Dataset` resource to train the model..
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `time_column`: Time-series column for the forecast model.
- `time_series_identifier_column`: ID column for the time-series column.

The run method when completed, returns the Vertex AI Model resource.

The execution of the training pipeline take up to one hour.

In [ ]:
model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    model_display_name=MODEL_DISPLAY_NAME,
)

### Review model evaluation scores

After your model training has finished, you can review the evaluation scores.

### Create a pipeline for model evaluation

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())
    print(model_evaluation)

## Create pipeline for model evaluation

Now, you run a Vertex AI BatchPrediction job and generate evaluations on its results. 

To do so, you create a Vertex AI pipeline using the components available from the [`google-cloud-pipeline-components`](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.17/index.html) Python package.

### Define the pipeline

While defining the flow of the pipeline, you get the model resource first. Then, you sample the provided source dataset for batch predictions and create a batch prediction job. The explanations are enabled while creating the batch prediction job to generate feature attributions. Once the batch prediction job is completed, you get the forecasting evaluation metrics and import them into your model resource.

The pipeline uses the following components:

- `GetVertexModelOp`: Gets a Vertex AI Model artifact. 
- `ModelBatchPredictOp`: Creates a Vertex AI Batch prediction job. and waits for it to complete.
- `ModelEvaluationForecastingOp` : Compute evaluation metrics on a trained model’s batch prediction results. Creates a dataflow job with Apache Beam and TFMA to compute evaluation metrics. Supports point forecasting and quantile forecasting for tabular data.  
- `ModelImportEvaluationOp`: Imports a model evaluation artifact to an existing Vertex AI Model with `ModelService.ImportModelEvaluation`. 

Learn more about [Google Cloud Pipeline Model Evaluation components](https://google-cloud-pipeline-components.readthedocs.io/en/google-cloud-pipeline-components-1.0.20/google_cloud_pipeline_components.experimental.evaluation.html).

In [ ]:
import kfp


@kfp.dsl.pipeline(name="vertex-evaluation-automl-tabular-forecast-pipeline")
def evaluation_automl_tabular_feature_attribution_pipeline(
    project: str,
    location: str,
    root_dir: str,
    prediction_type: str,
    model_name: str,
    target_column_name: str,
    batch_predict_bq_source_uris: str,
    batch_predict_instances_format: str,
    batch_predict_predictions_format: str = "jsonl",
    batch_predict_prediction_data_sample_size: int = 10000,
):

    from google_cloud_pipeline_components.aiplatform import ModelBatchPredictOp
    from google_cloud_pipeline_components.experimental.evaluation import (
        GetVertexModelOp, ModelEvaluationForecastingOp,
        ModelImportEvaluationOp)

    # Get the Vertex AI model resource
    get_model_task = GetVertexModelOp(model_resource_name=model_name)

    # Run Batch Explanations
    batch_predict_task = ModelBatchPredictOp(
        project=project,
        location=location,
        model=get_model_task.outputs["model"],
        job_display_name="liquor-data-model-registry-batch-predict-evaluation",
        bigquery_source_input_uri=batch_predict_bq_source_uris,
        instances_format=batch_predict_instances_format,
        predictions_format=batch_predict_predictions_format,
        gcs_destination_output_uri_prefix=root_dir,
    )

    # Run evaluation based on prediction type and feature attribution component.
    # After, import the model evaluations to the Vertex model.

    eval_task = ModelEvaluationForecastingOp(
        project=project,
        root_dir=root_dir,
        predictions_gcs_source=batch_predict_task.outputs["gcs_output_directory"],
        target_field_name="ground_truth_actual_sales_dollars",
        location=location,
        predictions_format=batch_predict_predictions_format,
        prediction_score_column="prediction.value",
    )

    ModelImportEvaluationOp(
        forecasting_metrics=eval_task.outputs["evaluation_metrics"],
        model=get_model_task.outputs["model"],
        dataset_type=batch_predict_instances_format,
    )

### Compile the pipeline

Next, compile the pipeline to a JSON file.

In [ ]:
compiler.Compiler().compile(
    pipeline_func=evaluation_automl_tabular_feature_attribution_pipeline,
    package_path="tabular_forecasting_pipeline.json",
)

### Define the parameters to run the pipeline

Specify the required parameters to run the pipeline.


In [ ]:
PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/timeseries{UUID}"

To run your pipeline, you pass the following parameters:

- `project`: Project ID.
- `location`: Region where the pipeline is run.
- `root_dir`: The GCS directory for keeping staging files and artifacts. A random subdirectory is created under the directory to keep job info for resuming the job in case of failure.
- `model_name`: Resource name of the trained AutoML Tabular Forecasting model.
- `target_column_name`: Name of the column to be used as the target for forecasting.
- `batch_predict_bq_source_uris`: BigQuery table uri of input instances for batch prediction.
- `batch_predict_instances_format`: Format of the input instances for batch prediction. Format used here is'**jsonl**'.
- `batch_predict_prediction_data_sample_size`: Size of the samples to be considered for batch prediction and evaluation.

In [ ]:
parameters = {
    "project": PROJECT_ID,
    "location": REGION,
    "root_dir": PIPELINE_ROOT,
    "prediction_type": "regression",
    "model_name": model.resource_name,
    "target_column_name": target_column,
    "batch_predict_bq_source_uris": f"bq://vertex-ai-dev.{DATASET_ID}.2021_sales_predict",
    "batch_predict_instances_format": "bigquery",
    "batch_predict_prediction_data_sample_size": 4,
}

## Run the pipline

Create a Vertex AI pipeline job using the following parameters:

- `display_name`: The name of the pipeline, this will show up in the Google Cloud console.
- `template_path`: The path of PipelineJob or PipelineSpec JSON or YAML file. It can be a local path, a Google Cloud Storage URI or an Artifact Registry URI.
- `parameter_values`: The mapping from runtime parameter names to its values that
        control the pipeline run.
- `enable_caching`: Whether to turn on caching for the run.

Learn more about the [PipelineJob class](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.PipelineJob).

After creating, run theVertex AI Pipeline Job using the configured `SERVICE_ACCOUNT`.

In [ ]:
DISPLAY_NAME = "forecasting" + UUID

job = aiplatform.PipelineJob(
    display_name=DISPLAY_NAME,
    template_path="tabular_forecasting_pipeline.json",
    parameter_values=parameters,
    enable_caching=False,
)

job.run(service_account=SERVICE_ACCOUNT)

## Model Evaluation

In the results from last step, click on the generated link to see your run in the Cloud Console.

In the UI, many of the pipeline directed acyclic graph (DAG) nodes expand or collapse when you click on them. Here is a partially-expanded view of the DAG (click image to see larger version).

<img src="images/automl_tabular_forecasting_evaluation_pipeline.png" style="height:600px;width:600px">

### Get the Model Evaluation Results

After the evalution pipeline is finished, run the below cell to print the evaluation metrics.

In [ ]:
from google.cloud import aiplatform_v1

# Iterate over the pipeline tasks
for task in job._gca_resource.job_detail.task_details:
    # Obtain the artifacts from the evaluation task
    if (
        ("model-evaluation" in task.task_name)
        and ("model-evaluation-import" not in task.task_name)
        and (
            task.state == aiplatform_v1.types.PipelineTaskDetail.State.SUCCEEDED
            or task.state == aiplatform_v1.types.PipelineTaskDetail.State.SKIPPED
        )
    ):
        evaluation_metrics = task.outputs.get("evaluation_metrics").artifacts[
            0
        ]  # ['artifacts']
        evaluation_metrics_gcs_uri = evaluation_metrics.uri

print(evaluation_metrics)
print(evaluation_metrics_gcs_uri)

### Visualize the metrics

Visualize the metrics using `matplotlib` Python package

In [ ]:
import matplotlib.pyplot as plt

metrics = []
values = []
for i in evaluation_metrics.metadata.items():
    if (
        i[0] == "meanAbsolutePercentageError"
    ):  # we are not considering MAPE as it is infinite. MAPE is infinite if groud truth is 0 as in our case Age is 0 for some instances.
        continue
    metrics.append(i[0])
    values.append(i[1])
plt.figure(figsize=(10, 5))
plt.bar(x=metrics, height=values)
plt.title("Evaluation Metrics")
plt.ylabel("Value")
plt.show()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Vertex AI Dataset
- AutoML Training Job
- Vertex AI Model
- Vertex AI Pipeline Job
- Cloud Storage Bucket  (set `delete_bucket` to **True** to delete your bucket)

In [ ]:
# Set dataset variable to the ID of the dataset to fetch.
bq_dataset = f"{PROJECT_ID}.{DATASET_ID}"

# Use the delete_contents parameter to delete a dataset and its contents.
# Use the not_found_ok parameter to not receive an error if the dataset has already been deleted.
client.delete_dataset(
    bq_dataset, delete_contents=True, not_found_ok=True
)  # Make an API request.

print("Deleted dataset '{}'.".format(bq_dataset))

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

# Delete dataset
dataset.delete()

# Training job
training_job.delete()

# Delete model
model.delete()

# Evaluation job
job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI